In [2]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe' # or 'notebook' or 'colab' or 'jupyterlab'

pd.set_option('display.max_columns', None)

In [18]:
df = pd.read_csv("iseller_data.csv")

df['order_date'] = pd.to_datetime(df['order_date'])
df['hour'] = df['order_date'].dt.hour
df['day'] = df['order_date'].dt.day

In [4]:
def filter_dataframe(df, date_start=None, date_end=None, product_category=None, product_name=None):
    filtered_df = df.copy()  # Create a copy to avoid modifying the original DataFrame
    
    # Apply filters based on the provided parameters
    if date_start is not None:
        filtered_df = filtered_df[filtered_df['order_date'] >= pd.to_datetime(date_start)]
    if date_end is not None:
        filtered_df = filtered_df[filtered_df['order_date'] <= pd.to_datetime(date_end)]
    if product_category is not None:
        filtered_df = filtered_df[filtered_df['product_category'] == product_category]
    if product_name is not None:
        filtered_df = filtered_df[filtered_df['product_name'] == product_name]
    
    return filtered_df



product_quantity = df.groupby('product_name')['quantity'].sum().reset_index().sort_values(by='quantity', ascending=False)

In [57]:
def plot_barchart(df, num_products, title):
    ## To do
    # Title
    # Proper display
    # Fix legends
    # X and Y label
    df = df.head(num_products)
    fig = px.bar(df, x='product_name', y='quantity', title=title,
                 labels={
                 "product_name": "Product Name",
                 "quantity": "Quantity",
              
             })
  
    return fig.show()

    

In [58]:
def get_total_product_sales(df, date_start=None, date_end =None, product_category=None, product_name=None):
    product_quantity = df.groupby('product_name')['quantity'].sum().reset_index()
    return product_quantity

In [59]:
plot_barchart(product_quantity, 20, title="Top Selling Items")

In [43]:
def plot_line_chart(df, group_by="hour"):
    agg_df = df.groupby(group_by)['quantity'].sum().reset_index()
    fig = px.line(agg_df, x=group_by, y='quantity')
    return fig.show()

In [60]:
plot_line_chart(df, "hour")

In [52]:
# def plot_piechart
fig = px.pie(product_quantity.head(10), values='quantity', names='product_name')
fig.show()


In [51]:
product_quantity.tail()

,product_name,quantity
77,Sababay Pink Blossom 750ml,1.0
78,Sababay White velvet 750ml,1.0
87,V24 - Apricot + Tonic Water,1.0
90,V24 - Blue Curacao + Ginger Ale,1.0
61,Konig Ludwig Botol 330ml,1.0
